In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
plt.rcParams['figure.max_open_warning'] = 0
import numpy as np
#import pandas as pd
df5 = pd.read_table(r"D:\Yuliya_test_data_table\Yuliya_test_data_table_deconv-only.txt", delimiter = '\t')#, nrows=2)


In [12]:
dfclean5 = df5.copy()
#keep this order because I went through the output file individually to determine these lines
dfclean5 = dfclean5[~dfclean5.index.isin(range(33482, 33797))] #122
dfclean5 = dfclean5[~dfclean5.index.isin(range(34747, 35652))] #126, 127, 128
dfclean5 = dfclean5[~dfclean5.index.isin(range(35930, 36501))] #130, 131
dfclean5 = dfclean5[~dfclean5.index.isin(range(36770, 37054))] #133
dfclean5 = dfclean5[~dfclean5.index.isin(range(38773, 39015))] #140
dfclean5 = dfclean5[~dfclean5.index.isin(range(39325, 39543))] #142
dfclean5 = dfclean5[~dfclean5.index.isin(range(40030, 40626))] #145, 146
dfclean5 = dfclean5[~dfclean5.index.isin(range(46853, 49718))] #170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182
dfclean5 = dfclean5[~(dfclean5.index >= 115849)] #the rest

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Disable column width wrapping

# Print the entire dataframe
#print(dfclean5)
#dfclean2.to_excel('C:/Users/Administrator/Desktop/dfclean5.xlsx', index=False)

In [1]:
groupedraw5= {}
dfclean5['plates'] = dfclean5['plate'].str.extract(r'(\d+)').astype(int)

for pattern, group in dfclean5.groupby('plates'):
    groupedraw5[pattern] = group
    
for key, df in groupedraw5.items():
    plt.figure(figsize=(25, 6))
    #plt.scatter(df.index, df['ratio RR'].astype(int), label='ratio RR')  
    plt.scatter(df.index, df['C15'].astype(int), label='C15')  
    plt.scatter(df.index, df['C25'].astype(int), label='C25')  
    plt.scatter(df.index, df['R15'].astype(int), label='R15')  

    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Dif. to R15')  # Convert to int and plot 'C25' column

    
   # plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
   # plt.ylabel('Data')
   # plt.title(f'Plot for plate {key}')
   # plt.legend()
   # plt.show()

NameError: name 'dfclean5' is not defined

In [19]:
Mclean5 = dfclean5.copy()

columns_to_check = ['C15', 'C25', 'D15', 'D25', 'R15', 'R25']


for column in columns_to_check:
    Mclean5[column] = Mclean5[column].apply(lambda x: x if x >= 200 else 0)
    
Mclean5 = Mclean5[~(Mclean5[columns_to_check] < 200).all(axis=1)]

    
#print(Mclean5)

In [20]:
adjusted5= Mclean5.copy()
numeric_columns = adjusted5.select_dtypes(include=[np.number]).columns

# Calculate z-scores and adjust outliers to the mean
numeric_columns = adjusted5.select_dtypes(include=[np.number]).columns

# Adjust outliers to the mean
for col in numeric_columns:
    mean = adjusted5[col].mean()
    std = adjusted5[col].std()
    z_scores = (adjusted5[col] - mean) / std
    
    # Replace outliers (threshold: 3 standard deviations from the mean) with the mean
    outlier_threshold = 20.0
    adjusted5[col] = np.where(np.abs(z_scores) > outlier_threshold, mean, adjusted5[col])
    adjusted5[col] =  adjusted5[col].astype(int)
#print(dfclean3_adjustedto_m)

In [21]:
Mclean5_scaled = adjusted5
columns_to_standardize = ['C15', 'C25', 'D15', 'D25', 'R15', 'R25']
Mclean5_scaled[columns_to_standardize] = scaler.fit_transform(Mclean5[columns_to_standardize])


martin wants: 

ratio of regular air to high co2 

+plot that against other replicate

+light vs dark


In [22]:
Mclean5_scaled['ratio RR'] =Mclean5_scaled['R15']//Mclean5_scaled['R25']
Mclean5_scaled['ratio C1R1'] =Mclean5_scaled['R15']//Mclean5_scaled['C15']
Mclean5_scaled['ratio C1R2'] =Mclean5_scaled['R15']//Mclean5_scaled['C25']
Mclean5_scaled['ratio C2R1'] =Mclean5_scaled['R25']//Mclean5_scaled['C15']
Mclean5_scaled['ratio C2R2'] =Mclean5_scaled['R25']//Mclean5_scaled['C25']

Mclean5_scaled['ratio D1R1'] =Mclean5_scaled['D15']//Mclean5_scaled['R15']
Mclean5_scaled['ratio D2R1'] =Mclean5_scaled['D25']//Mclean5_scaled['R15']
Mclean5_scaled = Mclean5_scaled.replace([np.inf, -np.inf], 0).dropna()


#print(Mclean5_scaled)

In [23]:
grouped5= {}

for pattern, group in Mclean5_scaled.groupby('plates'):
    grouped5[pattern] = group


In [2]:
for key, df in grouped5.items():
    plt.figure(figsize=(25, 6)) 
    #plt.scatter(df.index, df['ratio C1R1'].astype(int), label='ratio C1R1')  
    #plt.scatter(df.index, df['ratio C2R1'].astype(int), label='ratio C2R1')  
    plt.scatter(df.index, df['ratio C1R1'].astype(int), label='ratio R1C1')
    plt.scatter(df.index, df['ratio C2R2'].astype(int), label='ratio R2C2')
    
    plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
    plt.ylabel('Data')
    plt.title(f'Plot for plate {key}')
    plt.legend()
    plt.show()

NameError: name 'grouped5' is not defined

In [ ]:
Mclean5['ratio RR'] =Mclean5['R15']//Mclean5['R25']
Mclean5['ratio C1R1'] =Mclean5['C15']//Mclean5['R15']
Mclean5['ratio C1R2'] =Mclean5['C15']//Mclean5['R25']
Mclean5['ratio C2R1'] =Mclean5['C25']//Mclean5['R15']
Mclean5['ratio C2R2'] =Mclean5['C25']//Mclean5['R25']
Mclean5['plates'] = Mclean5['plate'].str.extract(r'(\d+)').astype(int)
Mclean5 = Mclean5.replace([np.inf, -np.inf], 0).dropna()


#print(Mclean5)

In [ ]:
grouped5f= {}

for pattern, group in Mclean5.groupby('plates'):
    grouped5f[pattern] = group

In [25]:
for key, df in grouped5f.items():
    plt.figure(figsize=(25, 6))
    #plt.scatter(df.index, df['ratio RR'].astype(int), label='ratio RR')  
    plt.scatter(df.index, df['ratio C1R1'].astype(int), label='ratio C1R1')  
    plt.scatter(df.index, df['ratio C2R1'].astype(int), label='ratio C2R1')  
    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Dif. to R15')  # Convert to int and plot 'C25' column  
    plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
    plt.ylabel('Data')
    plt.title(f'Plot for plate {key}')
    plt.legend()
    plt.show()

NameError: name 'grouped5f' is not defined

In [ ]:
for key, df in grouped5f.items():
    plt.figure(figsize=(25, 6))
    #plt.scatter(df.index, df['ratio RR'].astype(int), label='ratio RR')  
    plt.scatter(df.index, df['C15'].astype(int), label='C15')  
    plt.scatter(df.index, df['C25'].astype(int), label='C25')  
    plt.scatter(df.index, df['R15'].astype(int), label='R15')  

    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Dif. to R15')  # Convert to int and plot 'C25' column

    
  #  plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
   # plt.ylabel('Data')
   # plt.title(f'Plot for plate {key}')
    #plt.legend()
    #plt.show()

In [ ]:
one_wk = Mclean5.iloc[:150]  
two_wk = Mclean5.iloc[150:300]   
one_wkadj = adjusted5.iloc[:150]  
two_wkadj = adjusted5.iloc[150:300]   
one_wkraw = dfclean5.iloc[:150] 
two_wkraw= dfclean5.iloc[150:300] 

In [ ]:

columns_to_compare = ['C15', 'D25', 'C25', 'D15', 'R15', 'R25']

# Combine the data from all DataFrames for box plotting
all_dataone = [one_wk[col] for col in columns_to_compare]
all_datatwo = [two_wk[col] for col in columns_to_compare]
all_dataoneadj = [one_wkadj[col] for col in columns_to_compare]
all_datatwoadj = [two_wkadj[col] for col in columns_to_compare]
raw1 = [one_wkraw[col] for col in columns_to_compare]
raw2 = [two_wkraw[col] for col in columns_to_compare]

labels = columns_to_compare

fig, axs = plt.subplots(3, 2, figsize=(12, 15))
plt.subplots_adjust(hspace=0.4)

# Adjust vertical spacing between subplots

axs[0, 0].boxplot(all_dataone)
axs[0, 0].set_title('Box Plot for one_wk')
axs[0, 0].set_xticklabels(labels)
axs[0, 0].set_ylabel('Value')

axs[0, 1].boxplot(all_datatwo)
axs[0, 1].set_title('Box Plot for two_wk')
axs[0, 1].set_xticklabels(labels)
axs[0, 1].set_ylabel('Value')

axs[1, 0].boxplot(all_dataoneadj)
axs[1, 0].set_title('Box Plot for one_wkadj')
axs[1, 0].set_xticklabels(labels)
axs[1, 0].set_ylabel('Value')

axs[1, 1].boxplot(all_datatwoadj)
axs[1, 1].set_title('Box Plot for two_wkadj')
axs[1, 1].set_xticklabels(labels)
axs[1, 1].set_ylabel('Value')

axs[2, 0].boxplot(raw1)
axs[2, 0].set_title('Box Plot for one_wk raw')
axs[2, 0].set_xticklabels(labels)
axs[2, 0].set_ylabel('Value')

axs[2, 1].boxplot(raw2)
axs[2, 1].set_title('Box Plot for two_wk raw')
axs[2, 1].set_xticklabels(labels)
axs[2, 1].set_ylabel('Value')

plt.show()


In [ ]:
droppedR_df = Mclean5_scaled.copy()
droppedR_df['R_diff'] =droppedR_df['R15']-droppedR_df['R25']


droppedR_df = droppedR_df[droppedR_df['R_diff'] <1]
droppedR_df = droppedR_df[droppedR_df['R_diff'] >-1]
#print( droppedR_df)

In [ ]:
droppedR_df['ratio RR'] =droppedR_df['R15']//droppedR_df['R25']
droppedR_df['ratio C1R1'] =droppedR_df['C15']//droppedR_df['R15']
droppedR_df['ratio C1R2'] =droppedR_df['C15']//droppedR_df['R25']
droppedR_df['ratio C2R1'] =droppedR_df['C25']//droppedR_df['R15']
droppedR_df['ratio C2R2'] =droppedR_df['C25']//droppedR_df['R25']
droppedR_df['plates'] = droppedR_df['plate'].str.extract(r'(\d+)').astype(int)
droppedR_df = droppedR_df.replace([np.inf, -np.inf], 0).dropna()

In [24]:
droppedR_dfgroups = {}
for pattern, group in droppedR_df.groupby('plates'):
    droppedR_dfgroups[pattern] = group
#    print( group.columns)

NameError: name 'droppedR_df' is not defined

In [ ]:
for key, df in droppedR_dfgroups.items():
    plt.figure(figsize=(25, 6))
    #plt.scatter(df.index, df['ratio RR'].astype(int), label='ratio RR')  
    plt.scatter(df.index, df['ratio C1R1'].astype(int), label='ratio R1C1')  
    plt.scatter(df.index, df['ratio C2R1'].astype(int), label='ratio R1C1')  
    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Dif. to R15')  # Convert to int and plot 'C25' column

    
    plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
    plt.ylabel('Data')
    plt.title(f'Plot for plate {key}')
    plt.legend()
    plt.show()